In [ ]:
##Question 1: Hough Transform

functions used in the question are implemented in `q1_funcs` file:

## q1_funcs

The first function in this file is `generate_accumulator_array`,
which takes a binary image containing the edges of the image,
and two arrays containing the range of rho and theta in accumulator
matrix and outputs the matrix :

In [ ]:
import numpy as np
import cv2


def generate_accumulator_array(img_edge, rho_range, theta_range):
    result = np.zeros((rho_range.size, theta_range.size))
    offset = int(rho_range.size / 2)
    mask = np.where(img_edge > 0)
    for i in range(mask[0].size):
        y, x = mask[0][i], mask[1][i]
        for theta in theta_range:
            theta_rad = np.deg2rad(theta)
            rho = np.round(x * np.cos(theta_rad) + y * np.sin(theta_rad)) + offset
            result[int(rho), int(theta)] += 1

    return result

For each point on an edge, function iterates on different values of
theta and finds the corresponding rho, and votes the corresponding
cell in result matrix.

Function `draw_line_with_slope` takes an image,a value m as
line slope, a value b and a color and draws a line with given m and b
on the image :

In [ ]:
def draw_line_with_slope(img, m, b, color=(255, 0, 0)):
    height, width = img[:, :, 0].shape
    x1, x2, y1, y2 = int(-b / m), int((height - b) / m), int(b), int(m * width + b)
    if 0 <= x1 <= width and 0 <= y1 <= height:
        cv2.line(img, (x1, 0), (0, y1), color, 3)
    elif 0 <= x1 <= width and 0 <= y2 <= height:
        cv2.line(img, (x1, 0), (width, y2), color, 3)
    elif 0 <= x2 <= width and 0 <= y1 <= height:
        cv2.line(img, (x2, height), (0, y1), color, 3)
    elif 0 <= x2 <= width and 0 <= y2 <= height:
        cv2.line(img, (x2, height), (width, y2), color, 3)
    elif 0 <= x1 <= width and 0 <= x2 <= width:
        cv2.line(img, (x1, 0), (x2, height), color, 3)
    elif 0 <= y1 <= height and 0 <= y2 <= height:
        cv2.line(img, (0, y1), (width, y2), color, 3)

`draw_line` function takes an image and values rho and theta,
calculates corresponding slope and b, and uses the previous
function to draw a line on image (also outputs m and b):

In [ ]:
def draw_line(img, rho, theta):
    theta_rad = np.deg2rad(theta)
    sin, cos = np.sin(theta_rad), np.cos(theta_rad)
    if sin == 0:
        sin += 0.00000001
    if cos == 0:
        cos += 0.00000001
    m, b = -cos / sin, rho / sin
    draw_line_with_slope(img, m, b)
    return [m, b]

Function `draw_lines` uses accumulator array and a threshold to
draw lines on the image, using the functions implemented before.
it also outputs a matrix containing the slope and width of the
drawn lines :

In [ ]:
def draw_lines(accumulator_array, img, threshold, rho_offset):
    mask = np.where(accumulator_array > threshold)
    lines = np.zeros((2, 0))
    min_rho_diff, min_theta_diff = 30, 45
    prev_rho, prev_theta = int(mask[0][0]), int(mask[1][0])
    for i in range(mask[0].size):
        rho, theta = int(mask[0][i]), int(mask[1][i])
        if theta > 2 and np.abs(theta - 90) > 2:
            if np.abs(rho - prev_rho) > min_rho_diff or np.abs(theta - prev_theta) > min_theta_diff:
                [m, b] = draw_line(img, rho - rho_offset, theta)
                lines = np.c_[lines, [m, b]]
                prev_rho, prev_theta = rho, theta
    return lines

From here, some functions are implemented for filtering lines and
drawing corners. The first one is `find_cross` , which takes
as two lines as input and outputs the intersection :

In [ ]:
def find_cross(m1, b1, m2, b2):
    x = int((b2 - b1) / (m1 - m2))
    y = int(m1 * x + b1)
    return y, x


Next function takes four integer input ( which are the sums of
r,g,b channels of some points in the image ) and returns True
if at least one of the two face to face values have a rather small difference,
or at least two of the adjacent values have a rather large value. Notice
that if the colors are taken around one corner of a chess cell,
then at least one of these two conditions must hold.
This function is used to check whether a cross point is a valid point of
 chess or not :

In [ ]:
def is_colors_valid(color_l, color_r, color_u, color_d):
    dif1, dif2, dif3 = np.abs(int(color_l) - int(color_d)), np.abs(int(color_l) - int(color_r)), np.abs(
        int(color_l) - int(color_u))
    dif4, dif5, dif6 = np.abs(int(color_r) - int(color_d)), np.abs(int(color_r) - int(color_u)), np.abs(
        int(color_u) - int(color_d))
    if (dif2 < 50 or dif6 < 50) and (dif4 > 300 or dif5 > 300):
        return True

    return False

Function `is_point_valid` takes two points and the slope of the line
passing through the point and determines whether the point is valid
, using the function described above.

In [ ]:
def is_point_valid(img, x, y, m):
    height, width = img.shape
    offset = 15
    if offset <= x < width - offset and offset <= y < height - offset:
        xl, xr, xu, xd = x - offset, x + offset, x, x
        yl, yr, yu, yd = y, y, y - offset, y + offset
        color_l, color_r, color_u, color_d = img[yl, xl], img[yr, xr], img[yu, xu], img[yd, xd]
        print(x,y)
        return is_colors_valid(color_l, color_r, color_u, color_d)

    return False

Using the above function, we implement the function `is_line_valid` which takes
 a line as input and determines using the above functions, whether the line
 is valid or not , by checking if it contains a minimum number of valid points
 ( which we represent with limit )

In [ ]:
def is_line_valid(img_gray, m, b, lines, limit=8):
    counter = 0
    for i in range(lines.shape[1]):
        m2, b2 = lines[0, i], lines[1, i]
        if np.abs(m - m2) > 0.3:
            y, x = find_cross(m, b, m2, b2)
            if is_point_valid(img_gray, x, y, m):
                counter += 1
    if counter >= limit:
        return True
    return False

The code iterates on all the lines and finds their crossing point
with the given line, and checks whether these points are valid or not.
if the number of valid points on a line is bigger than a certain limit,
then line is considered as valid.

Finally, we are ready to implement function `filter_lines_by_cross_points`,
which takes the image, lines and a limit, and uses above functions to
determine valid lines. Function outputs a list containing the valid lines:



In [ ]:
def filter_lines_by_cross_points(img, lines, limit=8):
    valid_lines = np.zeros((2, 0))
    img_gray = np.sum(img, axis=2, keepdims=True).reshape((img.shape[0], img.shape[1])).astype('int')
    print(img_gray.shape)
    for i in range(lines.shape[1]):
        m, b = lines[0, i], lines[1, i]
        if is_line_valid(img_gray, m, b, lines, limit):
            valid_lines = np.c_[valid_lines, [m, b]]

    return valid_lines

The last two functions are used to draw lines and corners. Their
implementation is rather straightforward :

In [ ]:
def draw_lines_with_slope(img, lines, color=(255, 0, 0)):
    for i in range(lines.shape[1]):
        m, b = lines[0, i], lines[1, i]
        draw_line_with_slope(img, m, b, color)


def draw_corners(img, valid_lines):
    for i in range(valid_lines.shape[1]):
        m1, b1 = valid_lines[0, i], valid_lines[1, i]
        for j in range(valid_lines.shape[1]):
            m2, b2 = valid_lines[0, j], valid_lines[1, j]
            if np.abs(m1 - m2) > 0.1:
                y, x = find_cross(m1, b1, m2, b2)
                cv2.circle(img, (x, y), 5, (0, 255, 0), -1)

## Main code for problem

Finally, after implementing all required functions, we can put them
together and apply them for the given images:

In [ ]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import cv2
import q1_funcs

#### Question 1 : Hough Transform ####
import q1_funcs

input_paths = ["inputs/im01.jpg", "inputs/im02.jpg"]
output_paths = ["outputs/res01.jpg", "outputs/res02.jpg", "outputs/res03-hough-space.jpg"
    , "outputs/res04-hough-space.jpg", "outputs/res05-lines.jpg", "outputs/res06-lines.jpg",
                "outputs/res07-chess.jpg", "outputs/res08-chess.jpg",
                "outputs/res09-corners.jpg", "outputs/res10-corners.jpg"]

''' first image '''
im1 = cv2.imread(input_paths[0])
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)

After opening the image, we use Canny edge detection algorithm
to get the edges of the image:

In [ ]:
'''  detecting edges '''
img_edge_1 = cv2.Canny(im1, 350, 350)

number_of_good_points_limit = 3

# showing and saving
plt.imshow(img_edge_1, cmap="gray")
plt.savefig(output_paths[0])
plt.show()

Now we generate accumulator array from edges and save an image of it:

In [ ]:
'''  generating accumulator array '''
max_distance = int(np.ceil(np.sqrt(im1.shape[0] ** 2 + im1.shape[1] ** 2)))
rho_range = np.linspace(-int(max_distance), int(max_distance), max_distance * 2)
theta_range = np.linspace(0, 179, 180)
threshold = 100

accumulator_array1 = q1_funcs.generate_accumulator_array(img_edge_1, rho_range, theta_range)
plt.imshow(accumulator_array1)
plt.savefig(output_paths[2])
plt.show()

From here, we get the lines from accumulator array, draw them,
remove irrelevant ones and select corners using the functions
defined earlier :

In [ ]:
''' drawing lines '''
img_lines_1 = im1.copy()
lines1 = q1_funcs.draw_lines(accumulator_array1, img_lines_1, threshold, int(rho_range.size / 2))
q1_funcs.filter_lines_by_cross_points(img_lines_1, lines1)
plt.imshow(img_lines_1)
plt.savefig(output_paths[4])
plt.show()

''' removing irrelevant lines '''
img_valid_lines1 = copy.deepcopy(im1)
valid_lines = q1_funcs.filter_lines_by_cross_points(im1, lines1, number_of_good_points_limit)
q1_funcs.draw_lines_with_slope(img_valid_lines1, valid_lines, color=(0, 0, 255))
plt.imshow(img_valid_lines1)
plt.savefig(output_paths[6])
plt.show()

''' drawing corners '''
img_corners1 = im1.copy()
q1_funcs.draw_corners(img_corners1, valid_lines)
plt.imshow(img_corners1)
plt.savefig(output_paths[8])
plt.show()

Repeating all the above steps for the second image :

In [ ]:
''' second image '''

im2 = cv2.imread(input_paths[1])
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)

''' detecting edges '''
img_edge_2 = cv2.Canny(im2, 350, 350)

plt.imshow(img_edge_2, cmap='gray')
plt.savefig(output_paths[1])
plt.show()

'''  generating accumulator array '''
max_distance = int(np.ceil(np.sqrt(im2.shape[0] ** 2 + im2.shape[1] ** 2)))
rho_range = np.linspace(-int(max_distance), int(max_distance), max_distance * 2)

accumulator_array2 = q1_funcs.generate_accumulator_array(img_edge_2, rho_range, theta_range)
plt.imshow(accumulator_array2)
plt.savefig(output_paths[3])
plt.show()

''' drawing lines '''
img_lines_2 = im2.copy()
lines2 = q1_funcs.draw_lines(accumulator_array2, img_lines_2, threshold, int(rho_range.size / 2))
q1_funcs.filter_lines_by_cross_points(img_lines_2, lines2)
plt.imshow(img_lines_2)
plt.savefig(output_paths[5])
plt.show()

''' removing irrelevant lines '''
img_valid_lines2 = copy.deepcopy(im2)
valid_lines2 = q1_funcs.filter_lines_by_cross_points(im2, lines2, number_of_good_points_limit)
q1_funcs.draw_lines_with_slope(img_valid_lines2, valid_lines2, color=(0, 0, 255))
plt.imshow(img_valid_lines2)
plt.savefig(output_paths[7])
plt.show()

''' drawing corners '''
img_corners2 = im2.copy()
q1_funcs.draw_corners(img_corners2, valid_lines2)
plt.imshow(img_corners2)
plt.savefig(output_paths[9])
plt.show()


